In [4]:
import pandas as pd
import requests
import time
import math

# =API KEY DE GOOGLE MAPS ===
API_KEY = "API KEY"

# =DESTINO FIJO (UDA) ===
DESTINO = "-2.918554,-79.001596"

# =LEER ARCHIVO CSV ===
df = pd.read_csv("Profesores_Unificado.csv", sep=";", encoding="latin1")

# === ESTANDARIZAR NOMBRES DE COLUMNAS ===
df.columns = [col.strip().upper() for col in df.columns]

# =FUNCIONES DE CORRECCIÓN ===
def corregir_latitud(valor):
    try:
        valor = str(valor).replace('.', '').replace(',', '').strip()
        return float(valor[:2] + '.' + valor[2:])
    except:
        return None

def corregir_longitud(valor):
    try:
        valor = str(valor).replace('.', '').replace(',', '').strip()
        return float(valor[:3] + '.' + valor[3:])
    except:
        return None

# === APLICAR CORRECCIONES ===
df['LATITUD'] = df['LATITUD'].apply(corregir_latitud)
df['LONGITUD'] = df['LONGITUD'].apply(corregir_longitud)
df['ORIGEN_COORD'] = df['LATITUD'].astype(str) + ',' + df['LONGITUD'].astype(str)

# === PREPARAR LOTES ===
batch_size = 25
num_batches = math.ceil(len(df) / batch_size)

distancias_km = []
duraciones_min = []
valid_index = []

print("📡 Consultando distancias por lotes...")

for i in range(num_batches):
    batch_df = df.iloc[i*batch_size:(i+1)*batch_size].copy()
    origenes = '|'.join(batch_df['ORIGEN_COORD'])

    url = (
        f"https://maps.googleapis.com/maps/api/distancematrix/json"
        f"?origins={origenes}"
        f"&destinations={DESTINO}"
        f"&mode=driving"
        f"&key={API_KEY}"
    )

    try:
        resp = requests.get(url)
        data = resp.json()

        for j, element in enumerate(data['rows']):
            idx = batch_df.index[j]
            try:
                if element['elements'][0]['status'] == 'OK':
                    distancia = element['elements'][0]['distance']['value'] / 1000
                    duracion = element['elements'][0]['duration']['value'] / 60
                    if distancia > 0.8 and distancia < 30:
                        distancias_km.append(distancia)
                        duraciones_min.append(duracion)
                        valid_index.append(idx)
            except:
                continue

    except Exception as e:
        print(f"⚠️ Error en batch {i}: {e}")
        continue

    time.sleep(1.5)

# === GUARDAR RESULTADOS ===
df_validos = df.loc[valid_index].copy()  # 👉 conserva TODAS las columnas
df_validos['DISTANCIA_GOOGLE_KM'] = distancias_km
df_validos['DURACION_MIN'] = duraciones_min

df_descartados = df.drop(valid_index).copy()

# === EXPORTAR RESULTADOS ===
df_validos.to_excel("REPORTE_google_con_distanciap.xlsx", index=False)
df_descartados.to_excel("REPORTE_google_descartadosp.xlsx", index=False)

print("✅ Procesamiento finalizado:")
print(f"   - Registros válidos: {len(df_validos)}")
print(f"   - Registros descartados: {len(df_descartados)}")
print("📁 Archivos exportados correctamente.")


📡 Consultando distancias por lotes...
✅ Procesamiento finalizado:
   - Registros válidos: 1182
   - Registros descartados: 184
📁 Archivos exportados correctamente.
